In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from pysheds.grid import Grid
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import rasterio
import mplleaflet
import geopandas as gpd

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
grid = Grid.from_raster('/home/iiitmk/Desktop/GIS_repo/WatershedStreamNetworkDelimitationwithPythonandPysheds-20201231T091455Z-001/WatershedStreamNetworkDelimitationwithPythonandPysheds/Rst/LocalDem.tif', data_name='dem')

In [ ]:
def plotFigure(data, label, cmap='Blues'):
    plt.figure(figsize=(12,10))
    plt.imshow(data ,extent=grid.extent, cmap = cmap)
    plt.colorbar(label=label)
    plt.grid()

In [ ]:
# # Plot the raw DEM data
# fig, ax = plt.subplots(figsize=(12,10))
# plt.imshow(grid.dem, extent=grid.extent, cmap='cubehelix', zorder=1)
# plt.colorbar(label='Elevation (m)')
# plt.title('Digital elevation map')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')

In [ ]:
# elevDem=grid.dem[:-1,:-1]
plotFigure(grid.dem, 'Elevation (m)')

In [ ]:
dirmap = (64,  128,  1,   2,    4,   8,    16,  32)


In [ ]:
grid.flowdir(data='dem', out_name='dir' , dirmap=dirmap)


In [ ]:
plotFigure(grid.dir , 'Flowdirection' , 'viridis')

In [ ]:
# Specify pour point
# x, y = 39.4, 24.75
x, y = 285612.017, 2936416.682

# Delineate the catchment
grid.catchment(data='dir', x=x, y=y, dirmap=dirmap, out_name='catch',
               recursionlimit=15000, xytype='label', nodata_out=0)

In [ ]:
grid.clip_to('catch')

In [ ]:
demview = grid.view('dem', nodata=np.nan)

In [ ]:
plotFigure(demview , "elevation(m)")

In [ ]:
grid.to_raster(demview , 'demview.tif')

In [ ]:
grid.accumulation(data='catch', dirmap=dirmap ,pad_inplace=False, out_name='acc')

In [ ]:
accView = grid.view('acc', nodata=np.nan)
plotFigure(accView,"Cell Number",'PuRd')

In [ ]:
streams = grid.extract_river_network('catch','acc',threshold=200 , dirmap=dirmap)
# streams["features"][:2]

In [ ]:
streams

In [ ]:
def savedict(dic,file):
    f = open(file,'w')
    f.write(str(dic))
    f.close()
    

In [ ]:
savedict(streams , 'streams.geojson')

In [ ]:
streamnet = gpd.read_file('/home/iiitmk/Desktop/GIS_repo/streams.geojson')
streamnet.crs = {'init' :'epsg:32613'}

In [ ]:
# The polygonize argument defaults to the grid mask when no arguments are supplied
shapes = grid.polygonize()


# Plot catchment boundaries
fig, ax = plt.subplots(figsize=(6.5, 6.5))

for shape in shapes: 
    coords = np.asarray(shape[0]['coordinates'][0]) 
    ax.plot(coords[:,0], coords[:,1], color='cyan')
    
ax.set_xlim(grid.bbox[0], grid.bbox[2])
ax.set_ylim(grid.bbox[1], grid.bbox[3])
ax.set_title('Catchment boundary (vector)')
gpd.plotting.plot_dataframe(streamnet, None, cmap='Blues', ax=ax)


In [ ]:
ax = streamnet.plot()
mplleaflet.display(fig=ax.figure, crs=streamnet.crs, tiles='esri_aerial')

In [ ]:
# grid.fill_depressions(data='dem', out_name='filled')
# grid.resolve_flats(data='filled',out_name='resolved')
# grid.flowdir(data='resolved', out_name='dir')


In [ ]:
# grid.dir


In [ ]:
# grid.accumulation(data='dir', out_name='acc')

In [ ]:
# maxindex = np.where(grid.acc==np.max(grid.acc))
# grid.catchment(x = maxindex[0][0], y = maxindex[1][0],data = 'dir', out_name='catch', xytype='index')

In [ ]:
# grid.view('acc')